In [ ]:
!pip install diffusers transformers ftfy accelerate

In [ ]:
import torch
import requests
from PIL import Image
from io import BytesIO
from diffusers import StableDiffusionImg2ImgPipeline

#model_id = "runwayml/stable-diffusion-v1-5"
model_id = "nitrosocke/Ghibli-Diffusion"

device = "cuda"

pipe = StableDiffusionImg2ImgPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to(
    device
)

In [ ]:
import os

qfolder = "qr"
if not os.path.exists(qfolder):
  os.mkdir(qfolder)

folder = "outputs"
if not os.path.exists(folder):
  os.mkdir(folder)

In [ ]:
!pip install qrcode
!pip install pyngrok
!pip install flask_cors
!pip install requests
!pip install Flask

import qrcode
import datetime

now = datetime.datetime.now()
d = now.strftime('%Y%m%d%H%M%S')

In [ ]:
# QRコード作成
def createQr(qrText):
  qr = qrcode.QRCode()
  qr.add_data(qrText)
  qr.make()
  qrimg = qr.make_image()
  qrpath = f'qr/qr_{d}.png'
  qrimg.save(qrpath)

  return qrpath

In [ ]:
# 画像生成
def genImg(qrImg, prompt):
  qrImg.thumbnail((512, 512))
  n_prompt = "(worst quality:1.4), (low quality:1.4), (monochrome:1.3), NSFW"
  n_steps = 30

  # 元の画像をどのくらい参照するか
  denoising_strength=0.45

  image = pipe(
      prompt=prompt,
      negative_prompt=n_prompt,
      image=qrImg,
      strength=denoising_strength,
  #    num_inference_steps=n_steps,
      guidance_scale=7.5
      ).images[0]

  image.save(f'outputs/output_{d}.png')

  return image

In [ ]:
# Test
qr_data = "test test2"
prompt = "Ghibli style beautiful mountains and shooting stars at the night."

qrpath = createQr(qr_data)
qrimg = Image.open(qrpath).convert("RGB")
qrimg

image = genImg(qrimg, prompt)
image

In [ ]:
import os
import requests
import io
import json
from flask import Flask, send_file, render_template, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok, conf

# ngrokトークンを設定
conf.get_default().auth_token = "2iSGz2xdhjgsTOClek6eMrjpT6F_7msYimGkAoqEEbhwQcTQt"

# ngrokで公開するURL
NGROK_DOMAIN = "wallaby-more-pony.ngrok-free.app"

app = Flask(__name__)
CORS(app)

@app.route('/', methods=['POST'])
def index():
  json_data = request.json
  print(json_data)
  qrText = json_data["qrText"]
  imgPrompt = json_data["imgPrompt"]

  # QRコード生成
  qrpath = createQr(qrText)
  qrimg = Image.open(qrpath).convert("RGB")
  print("+++++++++++++++++++++++++++")

  # 画像生成
  image = genImg(qrimg, imgPrompt)
#  image
  print(" ========================== ")

  img_byte_array = io.BytesIO()
  image.save(img_byte_array, format='PNG')
  img_byte_array.seek(0)

	# IPFSへのファイルのピン留め（アップロード）
  url = "https://api.pinata.cloud/pinning/pinFileToIPFS"
  files = {'file': ('qrcode.png', img_byte_array, 'image/png')}
  headers = {
		'pinata_api_key': "ef4b2776bda587ee59df",
		'pinata_secret_api_key': "668688893b3ee64dcfdcc2b8ba801ecda95d916fdc307bd21f8d8351be86940c"
	}
  response = requests.post(url, files=files, headers=headers)

	# 一時ファイルを削除
  img_byte_array.close()

  getUrl = 'ipfs://' + response.json()["IpfsHash"]
  print(getUrl)

  return getUrl

if __name__ == "__main__":
  public_url = ngrok.connect(5000, hostname=NGROK_DOMAIN)
  print(f"ngrok URL: {public_url}")
  app.run(port=5000)